In [2]:
# A dictionary of movie critics and their ratings of a small
# set of movies
critics={'Lisa Rose': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.5,
'Just My Luck': 3.0, 'Superman Returns': 3.5, 'You, Me and Dupree': 2.5,'The Night Listener': 3.0},
'Gene Seymour': {'Lady in the Water': 3.0, 'Snakes on a Plane': 3.5,
'Just My Luck': 1.5, 'Superman Returns': 5.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 3.5},
'Michael Phillips': {'Lady in the Water': 2.5, 'Snakes on a Plane': 3.0,
'Superman Returns': 3.5, 'The Night Listener': 4.0},
'Claudia Puig': {'Snakes on a Plane': 3.5, 'Just My Luck': 3.0,
'The Night Listener': 4.5, 'Superman Returns': 4.0,
'You, Me and Dupree': 2.5},
'Mick LaSalle': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'Just My Luck': 2.0, 'Superman Returns': 3.0, 'The Night Listener': 3.0,
'You, Me and Dupree': 2.0},
'Jack Matthews': {'Lady in the Water': 3.0, 'Snakes on a Plane': 4.0,
'The Night Listener': 3.0, 'Superman Returns': 5.0, 'You, Me and Dupree': 3.5},
'Toby': {'Snakes on a Plane':4.5,'You, Me and Dupree':1.0,'Superman Returns':4.0}}

In [3]:
alpha=[1,2,3,4,5,6,7]
rangelen=len(alpha)
x=rangelen
while(x>1):
    for j in range(0,x-1):
        print(alpha[j],alpha[x-1])
    x-=1



1 7
2 7
3 7
4 7
5 7
6 7
1 6
2 6
3 6
4 6
5 6
1 5
2 5
3 5
4 5
1 4
2 4
3 4
1 3
2 3
1 2


In [4]:
critics['Lisa Rose']['Just My Luck']
#list expression
print([critics['Lisa Rose'][item] for item in critics['Lisa Rose'] if item in critics['Gene Seymour']])

[2.5, 3.5, 3.0, 3.5, 2.5, 3.0]


In [5]:
def get_dict_key_list(keypair):
    keylist=[]
    for key in critics:
        keylist.append(key)
    return (keylist)

In [6]:
#Euclidean distance
from math import sqrt
# Returns a distance-based similarity score for person1 and person2
def sim_distance(prefs,person1,person2):
    # Get the list of shared_items
    si={}
    for item in prefs[person1]:
        if item in prefs[person2]:
            si[item]=1
    # if they have no ratings in common, return 0
    if len(si)==0: return 0
    # Add up the squares of all the differences
    sum_of_squares=sum([pow(prefs[person1][item]-prefs[person2][item],2) for item in prefs[person1] if item in prefs[person2]])
    return 1/(1+sum_of_squares)

In [38]:
# Returns the Pearson correlation coefficient for p1 and p2
def sim_pearson(prefs,p1,p2):
    # Get the list of mutually rated items
    si={}
    for item in prefs[p1]:
        if item in prefs[p2]:si[item]=1
        # Find the number of elements
    n=len(si)
        # if they are no ratings in common, return 0
    if n==0: return 0
        # Add up all the preferences
    sum1=sum([prefs[p1][it] for it in si])
    sum2=sum([prefs[p2][it] for it in si])
        # Sum up the squares
    sum1Sq=sum([pow(prefs[p1][it],2) for it in si])
    sum2Sq=sum([pow(prefs[p2][it],2) for it in si])
        # Sum up the products
    pSum=sum([prefs[p1][it]*prefs[p2][it] for it in si])
        # Calculate Pearson score
    num=pSum-(sum1*sum2/n)
    den=sqrt((sum1Sq-pow(sum1,2)/n)*(sum2Sq-pow(sum2,2)/n))
    if den==0: return 0
    r=num/den
    return r
#print(sim_pearson(critics,'Lisa Rose','Gene Seymour'))

In [51]:
'''def iterate_over_dict_keys(dict_sequence,keylist):
    x=len(alpha)
    while(x>1):
        for j in range(0,x-1):
            print(sim_distance(dict_sequence,keylist[j],keylist[x-1]))
        x-=1
#the above all can be written ore replace by 
#[sim_distance(critics,keylist[0],other) for other in keylist if keylist[0]!=other]
def iterate_over_dict_keys_pearson(dict_sequence,keylist):
    x=len(alpha)
    while(x>1):
        for j in range(0,x-1):
            print(sim_pearson(dict_sequence,keylist[j],keylist[x-1]))
        x-=1
'''

keylist=get_dict_key_list(critics)
#iterate_over_dict_keys(critics,keylist)
#iterate_over_dict_keys_pearson(critics,keylist)

In [40]:
# Returns the best matches for person from the prefs dictionary.
# Number of results and similarity function are optional params.
def topMatches(prefs,person,n=5,similarity=sim_pearson):
    scores=[(similarity(prefs,person,other),other) for other in prefs if other!=person]
    # Sort the list so the highest scores appear at the top
    scores.sort( )
    scores.reverse( )
    return scores[0:n]

In [54]:
print(topMatches(critics,'Toby',n=5))
print(topMatches(critics,'Toby',n=5,similarity=sim_distance))


[(0.9912407071619299, 'Lisa Rose'), (0.9244734516419049, 'Mick LaSalle'), (0.8934051474415647, 'Claudia Puig'), (0.66284898035987, 'Jack Matthews'), (0.38124642583151164, 'Gene Seymour')]
[(0.3076923076923077, 'Mick LaSalle'), (0.2857142857142857, 'Michael Phillips'), (0.23529411764705882, 'Claudia Puig'), (0.2222222222222222, 'Lisa Rose'), (0.11764705882352941, 'Jack Matthews')]


In [60]:
# Gets recommendations for a person by using a weighted average
# of every other user's rankings
def getRecommendations(prefs,person,similarity=sim_pearson):
    totals={}
    simSums={}
    for other in prefs:
        # don't compare me to myself
        if other==person: continue

        sim=similarity(prefs,person,other)
        # ignore scores of zero or lower
        if sim<=0: continue
        for item in prefs[other]:
        # only score movies I haven't seen yet
            if item not in prefs[person] or prefs[person][item]==0:
                # Similarity * Score
                totals.setdefault(item,0)
                totals[item]+=prefs[other][item]*sim
                # Sum of similarities
                simSums.setdefault(item,0)
                simSums[item]+=sim
    # Create the normalized list
    rankings=[(total/simSums[item],item) for item,total in totals.items( )]
    # Return the sorted list
    rankings.sort( )
    rankings.reverse( )
    return rankings

In [61]:
print(getRecommendations(critics,'Toby'))
print(getRecommendations(critics,'Toby',similarity=sim_distance))

[(3.3477895267131017, 'The Night Listener'), (2.8325499182641614, 'Lady in the Water'), (2.530980703765565, 'Just My Luck')]
[(3.5002478401415877, 'The Night Listener'), (2.7561242939959363, 'Lady in the Water'), (2.461988486074374, 'Just My Luck')]


In [63]:
def transformPrefs(prefs):
    result={}
    for person in prefs:
        for item in prefs[person]:
            result.setdefault(item,{})
            # Flip item and person
            result[item][person]=prefs[person][item]
    return result
critics1=transformPrefs(critics)
print(topMatches(critics1,'Superman Returns',n=5))
print(topMatches(critics1,'Superman Returns',n=5,similarity=sim_distance))

[(0.6579516949597695, 'You, Me and Dupree'), (0.4879500364742689, 'Lady in the Water'), (0.11180339887498941, 'Snakes on a Plane'), (-0.1798471947990544, 'The Night Listener'), (-0.42289003161103106, 'Just My Luck')]
[(0.16666666666666666, 'Snakes on a Plane'), (0.10256410256410256, 'The Night Listener'), (0.09090909090909091, 'Lady in the Water'), (0.06451612903225806, 'Just My Luck'), (0.05333333333333334, 'You, Me and Dupree')]
